In [1]:
using DrWatson
@quickactivate

In [2]:
using BenchmarkTools, StaticArrays, Distributions
import ReverseDiff, ReverseDiff.DiffResults, Tracker, Zygote

# ReverseDiff with compiled tape
function get_gradient_reversediff(ℓπ, θ::AbstractVector)
    inputs = (θ,)
    f_tape = ReverseDiff.GradientTape(ℓπ, inputs)
    compiled_f_tape = ReverseDiff.compile(f_tape)
    results = similar.(inputs)
    all_results = DiffResults.GradientResult.(results)
    function ∂ℓπ∂θ(::Function, θ::AbstractVector)
        ReverseDiff.gradient!(all_results, compiled_f_tape, (θ,))
        return DiffResults.value(first(all_results)), DiffResults.gradient(first(all_results))
    end
    return ∂ℓπ∂θ
end

const GRADS = (
    :ReverseDiff => ReverseDiff.gradient,
    :ReverseDiffCompiled => get_gradient_reversediff,
    :Tracker => Tracker.gradient,
    :Zygote => Zygote.gradient,
)

;

In [3]:
function trybtime(f, x; verbose=true)
    try
        t = @belapsed $f($x)
        verbose && println(" --- takes $t seconds")
        return t
    catch e
        verbose && println(" --- fails due to $e")
        return NaN
    end
end

function benchmark(f, x; verbose=true)
    xs = [x, SVector{length(x), eltype(x)}(x)]
    res = []
    for x in xs
        verbose && print(rpad("$f(x::$(typeof(x)))", 60))
        trybtime(f, x; verbose=verbose)
        ts = []
        for grad in GRADS
            gradn, gradf = grad
            # Precompile the code
            if gradn == :ReverseDiffCompiled
                gradf = gradf(f, x)
            end
            verbose && print(rpad("- $gradn", 25))
            if x isa SArray && gradn == :Zygote
                verbose && println(" --- skips for SArray because it never finishs")
                continue
            end
            t = trybtime(x -> gradf(f, x), x; verbose=verbose)
            push!(ts, t)
        end
        push!(res, ts)
    end
    return res
end

;

## Simple loop

In [4]:
function logdensity(x)
    res = 0
    for i = 1:size(x, 1)
        res += x[i]
    end
    return res
end

macro unroll_logdensity(n)
    ex = :(function $(Symbol("logdensity_$n"))(x) end)
    body = ex.args[2]
    push!(body.args, :(res = 0))
    for i in 1:n
        push!(body.args, :(res += x[$i]))
    end
    push!(body.args, :(return res))
    return ex
end

@macroexpand(@unroll_logdensity(10)) |> display

;

:(function Main.logdensity_10(var"#104#x")
      #= In[4]:10 =#
      var"#103#res" = 0
      var"#103#res" += var"#104#x"[1]
      var"#103#res" += var"#104#x"[2]
      var"#103#res" += var"#104#x"[3]
      var"#103#res" += var"#104#x"[4]
      var"#103#res" += var"#104#x"[5]
      var"#103#res" += var"#104#x"[6]
      var"#103#res" += var"#104#x"[7]
      var"#103#res" += var"#104#x"[8]
      var"#103#res" += var"#104#x"[9]
      var"#103#res" += var"#104#x"[10]
      return var"#103#res"
  end)

In [5]:
let x = rand(500), logdensity_unroll = @unroll_logdensity(500)
    benchmark(logdensity, x)
    benchmark(logdensity_unroll, x)
end

;

logdensity(x::Array{Float64,1})                         --- takes 5.897888888888888e-7 seconds
- ReverseDiff             --- takes 0.000107142 seconds
- ReverseDiffCompiled     --- takes 1.9302e-5 seconds
- Tracker                 --- takes 0.000301681 seconds
- Zygote                  --- takes 0.001100816 seconds
logdensity(x::SArray{Tuple{500},Float64,1,500})         --- takes 4.646802030456853e-7 seconds
- ReverseDiff             --- takes 0.000111563 seconds
- ReverseDiffCompiled     --- takes 1.8827e-5 seconds
- Tracker                 --- fails due to ErrorException("setindex!(::SArray{Tuple{500},Float64,1,500}, value, ::Int) is not defined.")
- Zygote                  --- skips for SArray because it never finishs
logdensity_500(x::Array{Float64,1})                     --- takes 5.177631578947369e-7 seconds
- ReverseDiff             --- takes 0.000120347 seconds
- ReverseDiffCompiled     --- takes 1.8521e-5 seconds
- Tracker                 --- takes 0.000391226 seconds
- Zygote

In [6]:
let x = rand(1_000), logdensity_unroll = @unroll_logdensity(1_000)
    benchmark(logdensity, x)
    benchmark(logdensity_unroll, x)
end

;

logdensity(x::Array{Float64,1})                         --- takes 1.121e-6 seconds
- ReverseDiff             --- takes 0.000217781 seconds
- ReverseDiffCompiled     --- takes 3.614e-5 seconds
- Tracker                 --- takes 0.001065741 seconds
- Zygote                  --- takes 0.003784525 seconds
logdensity(x::SArray{Tuple{1000},Float64,1,1000})       --- takes 9.011627906976744e-7 seconds
- ReverseDiff             --- takes 0.000215418 seconds
- ReverseDiffCompiled     --- takes 3.7502e-5 seconds
- Tracker                 --- fails due to ErrorException("setindex!(::SArray{Tuple{1000},Float64,1,1000}, value, ::Int) is not defined.")
- Zygote                  --- skips for SArray because it never finishs
logdensity_1000(x::Array{Float64,1})                    --- takes 1.0478e-6 seconds
- ReverseDiff             --- takes 0.000253561 seconds
- ReverseDiffCompiled     --- takes 3.8117e-5 seconds
- Tracker                 --- takes 0.001173887 seconds
- Zygote                  --- 

## Stochastic volatility

In [7]:
# @model sto_volatility(y, ::Type{Tv}=Vector{Float64}) where {Tv} = begin
#     T = length(y)
#     ϕ ~ Uniform(-1, 1)
#     σ ~ truncated(Cauchy(0, 5), 0, Inf)
#     μ ~ Cauchy(0, 10)
#     h = Tv(undef, T)
#     h[1] ~ Normal(μ, σ / sqrt(1 - ϕ^2))
#     for t in 2:T
#         h[t] ~ Normal(μ + ϕ * (h[t-1] - μ), σ)
#     end
#     y ~ ArrayDist(Normal.(0, exp.(h / 2)))
# end

function rand_sv(T)
    ϕ = rand(Uniform(-1, 1))
    σ = rand(truncated(Cauchy(0, 5), 0, Inf))
    μ = rand(Cauchy(0, 10))
    h = zeros(T)
    h[1] = rand(Normal(μ, σ / sqrt(1 - ϕ^2)))
    for t in 2:T
        h[t] = rand(Normal(μ + ϕ * (h[t-1] - μ), σ))
    end
    y = rand.(Normal.(0, exp.(h / 2)))
    theta = [ϕ, σ, μ, h...]
    return (theta=theta, y=y)
end

function get_logdensity_sv(y)
    function logdensity_sv(theta)
        res = 0
        T = length(y)
        ϕ = theta[1]
        res += logpdf(Uniform(-1, 1), ϕ)
        σ = theta[2]
        res += logpdf(truncated(Cauchy(0, 5), 0, Inf), σ)
        μ = theta[3]
        res += logpdf(Cauchy(0, 10), μ)
        h = Vector{eltype(theta)}(undef, T)
        h[1] = theta[4]
        res += logpdf(Normal(μ, σ / sqrt(1 - ϕ^2)), h[1])
        for t in 2:T
            h[t] = theta[t+3]
            res += logpdf(Normal(μ + ϕ * (h[t-1] - μ), σ), h[t])
        end
        res += sum(logpdf.(Normal.(0, exp.(h / 2)), y))
        return res
    end
    return logdensity_sv
end

macro get_logdensity_unroll_sv(T)
    fn = Symbol("logdensity_sv_$T")
    getfn = Symbol("get_logdensity_sv_$T")
    ex = :(function $fn(theta) end)
    body = ex.args[2]
    push!(body.args, :(res = 0))
    push!(body.args, :(T = length(y)))
    push!(body.args, :(ϕ = theta[1]))
    push!(body.args, :(res += logpdf(Uniform(-1, 1), ϕ)))
    push!(body.args, :(σ = theta[2]))
    push!(body.args, :(res += logpdf(truncated(Cauchy(0, 5), 0, Inf), σ)))
    push!(body.args, :(μ = theta[3]))
    push!(body.args, :(res += logpdf(Cauchy(0, 10), μ)))
    push!(body.args, :(h = Vector{eltype(theta)}(undef, $T)))
    push!(body.args, :(h[1] = theta[4]))
    push!(body.args, :(res += logpdf(Normal(μ, σ / sqrt(1 - ϕ^2)), h[1])))
    for t in 2:T
        push!(body.args, :(h[$t] = theta[$t+3]))
        push!(body.args, :(res += logpdf(Normal(μ + ϕ * (h[$t-1] - μ), σ), h[$t])))
    end
    push!(body.args, :(res += sum(logpdf.(Normal.(0, exp.(h / 2)), y))))
    push!(body.args, :(return res))
    return :(function $getfn(y) $ex; return $fn end)
end

@macroexpand(@get_logdensity_unroll_sv(5)) |> display

;

:(function Main.get_logdensity_sv_5(var"#129#y")
      #= In[7]:73 =#
      function var"#128#logdensity_sv_5"(var"#136#theta")
          #= In[7]:54 =#
          var"#130#res" = 0
          var"#131#T" = Main.length(var"#129#y")
          var"#132#ϕ" = var"#136#theta"[1]
          var"#130#res" += Main.logpdf(Main.Uniform(-1, 1), var"#132#ϕ")
          var"#133#σ" = var"#136#theta"[2]
          var"#130#res" += Main.logpdf(Main.truncated(Main.Cauchy(0, 5), 0, Main.Inf), var"#133#σ")
          var"#134#μ" = var"#136#theta"[3]
          var"#130#res" += Main.logpdf(Main.Cauchy(0, 10), var"#134#μ")
          var"#135#h" = Main.Vector{Main.eltype(var"#136#theta")}(Main.undef, 5)
          var"#135#h"[1] = var"#136#theta"[4]
          var"#130#res" += Main.logpdf(Main.Normal(var"#134#μ", var"#133#σ" / Main.sqrt(1 - var"#132#ϕ" ^ 2)), var"#135#h"[1])
          var"#135#h"[2] = var"#136#theta"[2 + 3]
          var"#130#res" += Main.logpdf(Main.Normal(var"#134#μ" + var"#132#ϕ" * (var"#135#h"[

In [8]:
let res = rand_sv(50), get_logdensity_unroll_sv = @get_logdensity_unroll_sv(50)
    logdensity_sv = get_logdensity_sv(res.y)
    logdensity_sv_unroll = get_logdensity_unroll_sv(res.y)
    
    benchmark(logdensity_sv, res.theta)
    benchmark(logdensity_sv_unroll, res.theta)
end

;

logdensity_sv(x::Array{Float64,1})                      --- takes 1.8091e-6 seconds
- ReverseDiff             --- takes 0.000259718 seconds
- ReverseDiffCompiled     --- takes 4.5759e-5 seconds
- Tracker                 --- takes 0.000205208 seconds
- Zygote                  --- fails due to MethodError(Irrational{:log2π}, (1,), 0x0000000000006a2f)
logdensity_sv(x::SArray{Tuple{53},Float64,1,53})        --- takes 1.8591e-6 seconds
- ReverseDiff             --- takes 0.000257218 seconds
- ReverseDiffCompiled     --- takes 4.6164e-5 seconds
- Tracker                 --- fails due to ErrorException("setindex!(::SArray{Tuple{53},Float64,1,53}, value, ::Int) is not defined.")
- Zygote                  --- skips for SArray because it never finishs
#137#logdensity_sv_50(x::Array{Float64,1})              --- takes 1.8683e-6 seconds
- ReverseDiff             --- takes 0.000268866 seconds
- ReverseDiffCompiled     --- takes 4.4977e-5 seconds
- Tracker                 --- takes 0.000211031 second

In [9]:
let res = rand_sv(500), get_logdensity_unroll_sv = @get_logdensity_unroll_sv(500)
    logdensity_sv = get_logdensity_sv(res.y)
    logdensity_sv_unroll = get_logdensity_unroll_sv(res.y)
    
    benchmark(logdensity_sv, res.theta)
    benchmark(logdensity_sv_unroll, res.theta)
end

;

logdensity_sv(x::Array{Float64,1})                      --- takes 1.771e-5 seconds
- ReverseDiff             --- takes 0.003139698 seconds
- ReverseDiffCompiled     --- takes 0.00046833 seconds
- Tracker                 --- takes 0.003074429 seconds
- Zygote                  --- fails due to MethodError(Irrational{:log2π}, (1,), 0x0000000000006aaf)
logdensity_sv(x::SArray{Tuple{503},Float64,1,503})      --- takes 1.803e-5 seconds
- ReverseDiff             --- takes 0.002878644 seconds
- ReverseDiffCompiled     --- takes 0.00044813 seconds
- Tracker                 --- fails due to ErrorException("setindex!(::SArray{Tuple{503},Float64,1,503}, value, ::Int) is not defined.")
- Zygote                  --- skips for SArray because it never finishs
#173#logdensity_sv_500(x::Array{Float64,1})             --- takes 1.9161e-5 seconds
- ReverseDiff             --- takes 0.002964266 seconds
- ReverseDiffCompiled     --- takes 0.000450693 seconds
- Tracker                 --- takes 0.002784726 se